## Import OR-Tools (and more)

In [1]:
from ortools.linear_solver import pywraplp as OR
import pandas as pd
import math, itertools
import time
import random as rand

## Cereal Diet Problem (Lab 8)

$
\begin{align}
\min &\quad 3.8K + 4.2C \\
\text{subject to} &\quad 0.1K + 0.25C &\geq 1 \\
&\quad 1K + 0.25C &\geq 5 \\
&\quad 110K + 120C &\geq 400 \\
&\quad 0 \leq K, C
\end{align}
$

In [2]:
# define the model
diet = OR.Solver('diet', OR.Solver.GLOP_LINEAR_PROGRAMMING);

In [3]:
# decision variables
K = diet.NumVar(0, diet.infinity(), 'K');
C = diet.NumVar(0, diet.infinity(), 'C');

In [4]:
# objective function
diet.Minimize(3.8*K + 4.2*C);

In [5]:
# constraints
diet.Add(0.1*K + 0.25*C >= 1);
diet.Add(1*K + 0.25*C >= 5);
diet.Add(110*K + 120*C >= 400);

In [6]:
# solve and print solution
diet.Solve()
print('Objective =', diet.Objective().Value())
print('Solution:')
for v in diet.variables():
    print(v.name(),':', v.solution_value())

Objective = 26.222222222222225
Solution:
K : 4.444444444444445
C : 2.2222222222222228


## Stigler's Diet

*Based on CSV from http://excelcalculations.blogspot.com/2011/05/diet-problem-linear-programming.html

In [7]:
nutrients = pd.read_csv('nutrients.csv')
display(nutrients)
NUTRIENTS = []
bound = {}
for index, row in nutrients.iterrows():
    nutr = row['Nutrient']
    NUTRIENTS.append(nutr)
    bound.update({nutr : row['Bound']})

,Nutrient,Bound
0,Calories,1900
1,Fat,30
2,SatFat,0
3,TransFat,0
4,Cholesterol,0
5,Sodium,1500
6,Carbs,150
7,Fibre,20
8,Sugar,0
9,Protein,50


In [8]:
foods = pd.read_csv('foods.csv')
display(foods)
FOODS = []
cost = {}
amt = {}
for index, row in foods.iterrows():
    food = row['Food']
    FOODS.append(food)
    cost.update({food : row['Cost']})
    for nutr in NUTRIENTS:
        amt.update({(food, nutr) : row[nutr]})

,Food,Calories,Fat,SatFat,TransFat,Cholesterol,Sodium,Carbs,Fibre,Sugar,Protein,VitA,VitC,Calcium,Iron,Cost
0,Donut,239,11,3,0,18,232,30,1,12,4,13.8,0.7,27.6,2.2,0.85
1,Bagel,145,1,0,0,0,289,30,2,1,6,2.3,0.1,0.8,1.8,0.85
2,Yogurt,119,0,0,0,2,72,24,0,24,6,15.0,0.9,190.0,0.1,1.25
3,Chili,190,8,3,0,25,1040,17,5,5,14,1250.0,36.0,80.0,1.4,2.50
4,Brocolli,49,1,0,0,0,57,10,5,2,3,2167.0,90.9,56.0,0.9,0.40
5,Apple,95,0,0,0,0,2,25,4,19,0,98.3,8.4,10.9,0.2,0.40
6,Oats,105,2,0,0,0,72,19,3,0,4,1000.0,0.0,98.5,8.2,0.50
7,Orange,69,0,0,0,0,1,18,3,12,1,346.0,82.8,60.2,0.2,0.30
8,Lentils,116,0,0,0,0,2,20,8,2,9,8.0,1.5,19.0,3.3,0.35
9,Carrots,10,0,0,0,0,22,2,1,1,0,3861.0,0.7,9.0,0.2,0.30


In [9]:
# define model
m = OR.Solver('diet', OR.Solver.GLOP_LINEAR_PROGRAMMING)

In [10]:
# decision variables
x = {}
for i in FOODS:
    x[i] = m.NumVar(0, m.infinity(), ('%s' % (i)))

In [11]:
# objective function
m.Minimize(sum(x[i]*cost[i] for i in FOODS))

In [12]:
# subject to: edge capacities 
for j in NUTRIENTS:
    m.Add(sum(x[i]*amt[i,j] for i in FOODS) >= bound[j])

In [13]:
m.Solve()
print('Objective value =', m.Objective().Value())
print('Solution:')
for v in m.variables():
    print(v.name(),':', v.solution_value())

Objective value = 7.049928951302516
Solution:
Donut : 4.9606566996471955
Bagel : 0.0
Yogurt : 0.0
Chili : 0.0
Brocolli : 0.0
Apple : 0.0
Oats : 0.0
Orange : 2.6583746840246767
Lentils : 1.2608384583631382
Carrots : 0.0
BrusselSprots : 0.0
Chicken : 0.0
Blueberries : 0.0
Spinach : 2.709059022222989
Banana : 2.6061572810033424
Milk : 0.0


## Max Flow Implementation (Abstract Model)

In [14]:
# an abstract model
def MaxFlow(NODES,EDGES,capacity,source,sink):
    
    # define model
    m = OR.Solver('maxFlow', OR.Solver.GLOP_LINEAR_PROGRAMMING)
        
    # decision variables
    f = {}
    for i,j in EDGES:
        f[i,j] = m.NumVar(0, m.infinity(), ('(%s, %s)' % (i,j)))
            
    # objective function
    m.Maximize(sum(f[i,j] for i,j in EDGES if j == sink)) # set objective
            
    # subject to: edge capacities 
    for i,j in EDGES:
        m.Add(f[i,j] <= capacity[i,j])
            
    # subject to: flow conservation 
    for k in NODES:
        if k != source and k != sink:
            flowIn = sum(f[i,j] for i,j in EDGES if j == k)
            flowOut = sum(f[i,j] for i,j in EDGES if i == k)
            m.Add(flowIn == flowOut)
            
    return m

In [15]:
# EX: store max flow instance in two CSV files. Source and sink set manually.
nodes = pd.read_csv('nodes.csv') 
display(nodes)
edges = pd.read_csv('edges.csv') 
display(edges)

,nodes
0,1
1,2
2,3
3,4
4,5
5,6
6,7


,tail,head,capacity
0,1,2,1
1,1,3,7
2,2,3,5
3,2,4,6
4,2,5,2
5,3,5,2
6,3,6,4
7,4,7,5
8,5,4,1
9,5,6,4


In [16]:
# create node set
NODES = []
for index, row in nodes.iterrows():
    NODES.append(row['nodes'])
# set source and sink
source= 1
sink = 7

# create edge set
EDGES = []
capacity = {}
for index, row in edges.iterrows():
    edge = (row['tail'],row['head'])
    EDGES.append(edge)
    capacity.update({edge : row['capacity']})

In [17]:
# create a model from abstract MaxFlow model 
mod= MaxFlow(NODES,EDGES,capacity,source,sink)
# solve model and print results
mod.Solve()
print('Objective value =', mod.Objective().Value())
print('Solution:')
for v in mod.variables():
    print(v.name(),':', v.solution_value())

Objective value = 6.0
Solution:
(1, 2) : 1.0
(1, 3) : 5.0
(2, 3) : 0.0
(2, 4) : 0.0
(2, 5) : 1.0
(3, 5) : 2.0
(3, 6) : 3.0
(4, 7) : 0.0
(5, 4) : 0.0
(5, 6) : 0.0
(5, 7) : 3.0
(6, 7) : 3.0


## Branch and Bound (Not working yet)

$
\begin{align}
\max &\quad 5x_1 + 8x_2 \\
\text{subject to} &\quad x_1 + x_2 &\leq 6 \\
&\quad 5x_1 + 9x_2 &\leq 45 \\
&\quad x_1, x_2 &\leq 0 \\
&\quad x_1, x_2 &\text{integer} 
\end{align}
$

In [18]:
# define the model
lp = OR.Solver('lp', OR.Solver.GLOP_LINEAR_PROGRAMMING);
# decision variables
x1 = lp.NumVar(0, diet.infinity(), 'x1');
x2 = lp.NumVar(0, diet.infinity(), 'x2');
# objective function
lp.Maximize(5*x1 + 8*x2);
# constraints
lp.Add(x1 + x2 <= 6);
lp.Add(5*x1 + 9*x2 <= 45);

In [19]:
class Node:
    def __init__(self, lp, sol, z):
        self.lp = lp
        self.sol = sol
        self.z = z
    def lp(self):
        return lp
    def sol(self):
        return sol
    def z(self):
        return z 

In [20]:
# returns a dictionary of variables to values
def Solution(lp):
    vals = {}
    for v in lp.variables():
        vals.update({v : v.solution_value()})
    return vals

In [21]:
# branch on the given LP
def Branch(node):
    sol = node.sol
    for x in sol:
        val = sol[x]
        if not round(val,9).is_integer():
            branch = []
            leftLP = node.lp # not independent duplicate
            leftLP.Add(x <= math.floor(val)) 
            leftLP.Solve()
            if leftLP.FEASIBLE:
                zLeft = leftLP.Objective().Value()
                solLeft = Solution(leftLP)
                leftNode = Node(leftLP,solLeft,zLeft)
                branch.append(leftNode)                
            rightLP = node.lp
            rightLP.Add(x <= math.floor(val)) 
            rightLP.Solve()
            if rightLP.FEASIBLE:
                zRight = rightLP.Objective().Value()
                solRight = Solution(rightLP)
                rightNode = Node(rightLP,solRight,zRight)   
                branch.append(rightNode)
            return branch
    # solution is integral
    return node    

In [22]:
# takes the LP relaxation and returns optimal integral solution
def BranchAndBound(lp):
    
    lp.Solve()
    z = lp.Objective().Value()
    sol = Solution(lp)
    root= Node(lp,sol,z)
    
    firstIncumbent = True
    incumbent = None
    active = [root]
    
    while len(active) > 0:
        # get an active subproblem
        subproblem = active.pop()
        
        if firstIncumbent == False and subproblem.z < incumbent.z:
            continue
        
        # branch
        branch = Branch(subproblem)
        if branch == subproblem:
            # must be integral
            # see if incumbent should be updated
            if firstIncumbent or subproblem.z > incumbent.z:
                firstIncumbent = False
                incumbent = subproblem
        else:
            for n in branch:  
                active.append(n)
    
    return incumbent

In [23]:
opt = BranchAndBound(lp)
print('Objective: ', opt.z)
print('Solution:')
for x in opt.sol:
    print(x.name(), ':', opt.sol[x])

Objective:  34.0
Solution:
x1 : 2.0
x2 : 3.0


## Hotel Room Assignment (2019)

*Based on research Sam and I have worked on since Spring 2019

In [24]:
hotel = pd.read_csv('hotel.csv')
display(hotel.head())
ROOMS = []
rType = {}
for index, row in hotel.iterrows():
    room = row['number'].astype(int)
    ROOMS.append(room)
    rType.update({room : row['type']})

,number,type,quality,checkout,process
0,1,1,0.866501,132,5
1,2,2,0.722858,140,6
2,3,2,0.813548,113,6
3,4,3,0.773847,132,7
4,5,1,0.888199,130,4


In [25]:
arrivals = pd.read_csv('arrivals.csv')
display(arrivals.head())
GUESTS = []
gType = {}
for index, row in arrivals.iterrows():
    guest = row['id']
    GUESTS.append(guest)
    gType.update({guest : row['type']})

,id,type,arrival
0,1,1,193
1,2,1,183
2,3,1,180
3,4,1,182
4,5,1,178


In [26]:
weights = pd.read_csv('weights.csv')
display(weights.head())
weight = {}
for index, row in weights.iterrows():
    for r in ROOMS:
        weight.update({(row['guest'].astype(int),r): row[r]})

,guest,1,2,3,4,5,6,7,8,9,...,191,192,193,194,195,196,197,198,199,200
0,1,0.59469,0.61369,0.61830,0.58173,0.60438,0.62779,0.51696,0.56866,0.63766,...,0.54750,0.55342,0.59206,0.64322,0.57772,0.58002,0.62830,0.56761,0.52951,0.56736
1,2,0.98590,0.99859,1.00000,0.99919,0.99816,0.99160,0.89249,0.99482,0.98266,...,0.94115,0.94338,0.97669,1.00000,0.97370,0.99912,1.00000,0.94125,0.94520,0.98792
2,3,0.81001,0.79416,0.85622,0.88880,0.75852,0.89135,0.78734,0.83339,0.84954,...,0.80208,0.78426,0.80267,0.89909,0.86798,0.79901,0.84767,0.79826,0.76044,0.83412
3,4,0.95983,1.00000,0.97640,0.99053,0.98856,0.98552,0.95642,0.98620,0.96995,...,0.97711,1.00000,0.98429,1.00000,0.99151,1.00000,0.95541,1.00000,0.98179,1.00000
4,5,0.95033,0.94514,0.98076,0.94660,0.93321,0.98684,0.80125,0.99449,0.94104,...,0.80957,0.92215,0.91263,0.95574,0.97488,0.96240,0.95152,0.90602,0.88908,0.94013


In [27]:
PAIRS = list(itertools.product(GUESTS, ROOMS))

In [28]:
# solver
m = OR.Solver('lp', OR.Solver.CBC_MIXED_INTEGER_PROGRAMMING);

In [29]:
# decision variables
x = {}
for g,r in PAIRS:
    x[g,r] = m.IntVar(0,1,('(%s, %s)' % (g,r)))

In [30]:
# objective function
m.Maximize(sum(x[g,r]*weight[g,r] for g,r in PAIRS)/len(GUESTS)) # set objective

In [31]:
# subject to: every guest assigned one room
for g in GUESTS:
    m.Add(sum(x[g,r] for r in ROOMS) == 1)

In [32]:
# subject to: every room assigned at most one guest
for r in ROOMS:
    m.Add(sum(x[g,r] for g in GUESTS) <= 1)

In [33]:
# subject to: every guest assigned a room of appropriate type
for g in GUESTS:
    m.Add(sum(x[g,r]*rType[r] for r in ROOMS) >= gType[g])

In [ ]:
# subject to: 

In [34]:
m.Solve()
print('Objective value =', m.Objective().Value())
print('Solution:')
for g,r in PAIRS:
    val = x[g,r].solution_value()
    if val == 1:
        print(x[g,r].name(),':',weight[g,r])
m.Objective().BestBound()

Objective value = 0.9711755737704925
Solution:
(1, 171) : 0.65065
(2, 196) : 0.9991200000000001
(3, 93) : 0.8986700000000001
(4, 177) : 1.0
(5, 71) : 1.0
(6, 3) : 0.98892
(7, 96) : 0.9980600000000001
(8, 57) : 0.8096899999999999
(9, 53) : 0.9362799999999999
(10, 70) : 1.0
(11, 132) : 0.9975
(12, 59) : 1.0
(13, 104) : 0.93811
(14, 91) : 0.9220700000000001
(15, 51) : 0.89915
(16, 6) : 0.93422
(17, 149) : 0.93758
(18, 95) : 0.9988
(19, 88) : 1.0
(20, 194) : 1.0
(21, 9) : 0.92595
(22, 55) : 1.0
(23, 8) : 1.0
(24, 23) : 0.9121100000000001
(25, 34) : 1.0
(26, 89) : 1.0
(27, 120) : 0.9764700000000001
(28, 102) : 0.96516
(29, 159) : 0.82091
(30, 54) : 1.0
(31, 76) : 0.9831200000000001
(32, 1) : 0.9744799999999999
(33, 160) : 0.8335600000000001
(34, 32) : 1.0
(35, 191) : 1.0
(36, 164) : 1.0
(37, 64) : 1.0
(38, 151) : 0.99095
(39, 65) : 0.99419
(40, 152) : 1.0
(41, 137) : 1.0
(42, 108) : 1.0
(43, 19) : 0.99671
(44, 148) : 0.99223
(45, 195) : 0.9387200000000001
(46, 86) : 0.92066
(47, 114) : 1.0


0.9711755737704925

## Large Random LP (Setting Time Limits)

In [35]:
# runs the random LP with n constraints, m variables, and time limit t (ms)
def RandomLP(n,m,t):
    
    # set random seed
    rand.seed(1101)
    
    # define model
    mod = OR.Solver('rand', OR.Solver.GLOP_LINEAR_PROGRAMMING)
    mod.SetTimeLimit(t)
        
    # decision variables
    x = {}
    for i in range(m):
        x[i] = mod.NumVar(0, mod.infinity(), ('%s' % (i)))
            
    # objective function
    mod.Objective().SetMinimization()
    for i in range(m):
        mod.Objective().SetCoefficient(x[i],rand.random())
    
    # constraints
    for i in range(n):
        b = rand.random()
        constraint= mod.Constraint(b,mod.infinity())
        for j in range(m):
            constraint.SetCoefficient(x[j],rand.random())
            
    if (mod.FEASIBLE == 1):
        mod.Solve()
        print('Time', (t/1000),'second limit')
        print('Objective =', mod.Objective().Value())
        print('simplex iterations :', mod.iterations())
        print('branch-and-bound nodes :',mod.nodes())
        print()
    else:
        print('infeasible')

In [38]:
for t in [5000,4000,3000,2000,1500,1400,1300,1200]:
    RandomLP(2000,2500,t)

Time 5.0 second limit
Objective = 0.007380338377045288
simplex iterations : 531
branch-and-bound nodes : -1

Time 4.0 second limit
Objective = 0.015731857927706643
simplex iterations : 477
branch-and-bound nodes : -1

Time 3.0 second limit
Objective = 0.07218943486698329
simplex iterations : 300
branch-and-bound nodes : -1

Time 2.0 second limit
Objective = 0.2761895077162174
simplex iterations : 126
branch-and-bound nodes : -1

Time 1.5 second limit
Objective = 1.5336782559206978
simplex iterations : 32
branch-and-bound nodes : -1

Time 1.4 second limit
Objective = 1.2967199050029568
simplex iterations : 42
branch-and-bound nodes : -1

Time 1.3 second limit
Objective = 0.0
simplex iterations : 30
branch-and-bound nodes : -1

Time 1.2 second limit
Objective = 0.0
simplex iterations : 14
branch-and-bound nodes : -1

